In [12]:
# ⚠️ Esto reinicia completamente el entorno de ejecución de Colab
# Vas a perder todas las variables y archivos cargados.
import IPython
IPython.Application.instance().kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [15]:
!pip install requests tqdm

In [16]:
import os
import requests
import time
from tqdm import tqdm
import pandas as pd
from IPython.display import display

# === CONFIGURACIÓN GENERAL ===
PLATE_RECOGNIZER_TOKEN = "8e76af2fd4e4f6a34ec2ac6f291f18068823d1ce"  # 👈 reemplazá con tu token real
REPO_URL = "https://github.com/juanbohner/TPP---Radar-de-Tramo.git"
CARPETA_REPO = "TPP---Radar-de-Tramo"
CARPETA_IMAGENES = os.path.join(CARPETA_REPO, "Prueas con Plate Recognizer\Ejemplos de patentes")


In [25]:
# Clonar el repositorio (si no está ya clonado)
if not os.path.exists(CARPETA_REPO):
    !git clone {REPO_URL}

# Verificar que la carpeta exista
if not os.path.exists(CARPETA_IMAGENES):
    raise FileNotFoundError(f"No se encontró la carpeta {CARPETA_IMAGENES}")

# Cargar los nombres de las imágenes (solo .jpg y .png)
imagenes = [f for f in os.listdir(CARPETA_IMAGENES) if f.lower().endswith((".jpg", ".png"))]

print(f"📂 Se encontraron {len(imagenes)} imágenes:")
for img in imagenes:
    print("  -", img)


📂 Se encontraron 5 imágenes:
  - AC530BE.jpg
  - AE159KJ.jpg
  - AD822PW.png
  - AD030AC.jpg
  - ANO433.png


In [26]:
import time
import requests

def reconocer_patente(imagen_path, delay_entre_requests=2):
    """
    Envía una imagen a la API de Plate Recognizer y devuelve toda la información relevante.
    - Incluye región 'ar'
    - Gestiona reintentos si hay throttling
    - Devuelve información estructurada del resultado
    """
    url_api = "https://api.platerecognizer.com/v1/plate-reader/"
    headers = {"Authorization": f"Token {PLATE_RECOGNIZER_TOKEN}"}
    data = {"regions": ["ar"]}  # 🇦🇷 indica que las patentes son argentinas

    with open(imagen_path, "rb") as img_file:
        inicio = time.time()
        response = requests.post(url_api, headers=headers, data=data, files={"upload": img_file})
        fin = time.time()
        duracion = fin - inicio

    # Manejo de errores y throttling
    if response.status_code == 429:
        print("⏱️ Límite de velocidad alcanzado. Esperando más tiempo...")
        time.sleep(delay_entre_requests + 2)
        return reconocer_patente(imagen_path, delay_entre_requests + 2)

    if response.status_code != 201:
        return {
            "error": f"HTTP {response.status_code}",
            "texto": response.text,
            "tiempo": duracion
        }

    # Procesar respuesta
    data_json = response.json()
    results = data_json.get("results", [])

    if not results:
        return {
            "error": "Sin detección",
            "response_raw": data_json,
            "tiempo": duracion
        }

    best = results[0]
    plate = best.get("plate", "").upper()
    score = best.get("score", None)
    candidates = [c["plate"].upper() for c in best.get("candidates", [])]

    vehicle = best.get("vehicle", {})
    vehicle_type = vehicle.get("type", None)
    vehicle_score = vehicle.get("score", None)
    vehicle_box = vehicle.get("box", None)

    return {
        "plate": plate,
        "score": score,
        "candidates": candidates,
        "vehicle_type": vehicle_type,
        "vehicle_score": vehicle_score,
        "vehicle_box": vehicle_box,
        "response_raw": data_json,
        "tiempo": duracion
    }


In [27]:
import os
from tqdm import tqdm

# Asumimos que ya existen las variables:
# - PLATE_RECOGNIZER_TOKEN
# - CARPETA_IMAGENES
# - imagenes

resultados = {}

for nombre in tqdm(imagenes, desc="Procesando imágenes con Plate Recognizer"):
    ruta = os.path.join(CARPETA_IMAGENES, nombre)
    patente_real = os.path.splitext(nombre)[0].upper()

    r = reconocer_patente(ruta)

    resultados[nombre] = {
        "patente_real": patente_real,
        "plate_api": r.get("plate"),
        "score_api": r.get("score"),
        "candidatos": r.get("candidates"),
        "vehicle_type": r.get("vehicle_type"),
        "vehicle_score": r.get("vehicle_score"),
        "vehicle_box": r.get("vehicle_box"),
        "tiempo": r.get("tiempo"),
        "estado": "OK" if "error" not in r else r["error"],
        "json_raw": r.get("response_raw")
    }

    # Espera entre requests
    time.sleep(2)


Procesando imágenes con Plate Recognizer: 100%|██████████| 5/5 [00:12<00:00,  2.48s/it]


In [28]:
import pandas as pd
from IPython.display import display

data = []
for nombre, info in resultados.items():
    patente_real = info["patente_real"]
    plate_api = info.get("plate_api", "-")
    score_api = info.get("score_api", "-")
    match = (plate_api == patente_real)
    candidatos = ", ".join(info.get("candidatos", [])) if info.get("candidatos") else "-"

    data.append({
        "imagen": nombre,
        "patente_real": patente_real,
        "lectura_api": plate_api,
        "match": match,
        "score_api": score_api,
        "candidatos": candidatos,
        "tipo_vehiculo": info.get("vehicle_type", "-"),
        "score_vehiculo": info.get("vehicle_score", "-"),
        "tiempo_procesamiento (s)": round(info.get("tiempo", 0), 2),
        "estado": info.get("estado", "-")
    })

df_resultados = pd.DataFrame(data)
display(df_resultados)


,imagen,patente_real,lectura_api,match,score_api,candidatos,tipo_vehiculo,score_vehiculo,tiempo_procesamiento (s),estado
0,AC530BE.jpg,AC530BE,C630BF,False,0.914,"C630BF, C530BF, C6308F, C63OBF, C5308F, C53OBF...",SUV,0.928,0.67,OK
1,AE159KJ.jpg,AE159KJ,AE159KJ,True,0.998,AE159KJ,Sedan,0.681,0.44,OK
2,AD822PW.png,AD822PW,AD822PW,True,0.998,AD822PW,Sedan,0.927,0.36,OK
3,AD030AC.jpg,AD030AC,AD030AC,True,0.999,AD030AC,Sedan,0.947,0.37,OK
4,ANO433.png,ANO433,ANO433,True,0.994,ANO433,Pickup Truck,0.431,0.54,OK


In [29]:
# Calcular precisión general
if len(df_resultados) > 0:
    precision = df_resultados["match"].sum() / len(df_resultados)
    print(f"✅ Precisión total: {precision * 100:.1f}%")
else:
    print("⚠️ No hay datos para calcular precisión.")

# Guardar resultados en CSV
nombre_csv = "resultados_plate_recognizer_completo.csv"
df_resultados.to_csv(nombre_csv, index=False)
print(f"📁 Resultados guardados en '{nombre_csv}'")


✅ Precisión total: 80.0%
📁 Resultados guardados en 'resultados_plate_recognizer_completo.csv'


In [30]:
import json

with open("resultados_raw.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=2)

print("🧾 Archivo 'resultados_raw.json' guardado con los datos completos de la API.")


🧾 Archivo 'resultados_raw.json' guardado con los datos completos de la API.
